In [10]:
from Bio import SeqIO
import pandas as pd
import numpy as np
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord

In [2]:
#load family label file 
fl = pd.read_csv('famlabel.csv')
fl = fl.loc[fl['Fam'].notnull()] #1 missing entry had a NaN value

#1.gb
eyed, desc, tax = [],[],[]
for i in SeqIO.parse('1.gb', 'genbank'):
    eyed.append(i.id)
    desc.append(i.description)
    tax.append(i.annotations['taxonomy'])

df = pd.DataFrame({'Id': eyed, 'Des' : desc, 'Tax' : tax})
df = df.drop_duplicates(subset='Id')

for i,r in df.iterrows():
    df.loc[i,'Tax'] = ''.join(r['Tax'])

In [8]:
c = 0

for i in df['Tax'].to_list():
    if 'idae' not in i:
        print(i)
        c+=1
        
print(c)

#2688 weird entries hoping to be dropped during merging

VirusesRiboviriaunclassified Riboviriaunclassified ssRNA virusesunclassified ssRNA positive-strand viruses
VirusesRiboviriaNidoviralesunclassified Nidovirales
VirusesRiboviriaNidoviralesunclassified Nidovirales
Virusesunclassified virusesunclassified DNA virusesunclassified ssDNA viruses
Virusesunclassified viruses
VirusesRiboviriaunclassified Riboviriaunclassified ssRNA virusesunclassified ssRNA positive-strand viruses
VirusesRiboviriaunclassified Riboviriaunclassified ssRNA virusesunclassified ssRNA positive-strand viruses
VirusesRiboviriaBotybirnavirusunclassified Botybirnavirus
VirusesRiboviriaBotybirnavirusunclassified Botybirnavirus
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
Virusesunclassified viruses
Virusesunclassified viruses
Virusesunclassified viruses
Virusesunclassified viruses
Virusesunclassified viruses
Virusesunclassified viruses
Virusesunclassified viruses
V

VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassified Riboviria
VirusesRiboviriaunclassif

In [20]:
df['Gen'] = 'unk'

df = df.reset_index(drop = True)

for i,r in fl.iterrows():
    for i2,r2 in df.iterrows():
        if r['Fam'].lower() in r2['Tax'].lower():
            df.loc[i2, 'Gen'] = r['Genome type'] 

In [12]:
df.loc[df['Gen'] == 'unk']

#2693 left unlabelled

,Id,Des,Tax,Gen
3121,LC504569.1,"Goutanap virus 16GH1 genomic RNA, complete genome",VirusesRiboviriaunclassified Riboviriaunclassi...,unk
3135,MK182566.1,"Morelia viridis nidovirus isolate BH128/14-12,...",VirusesRiboviriaNidoviralesunclassified Nidovi...,unk
3136,MK182569.1,"Morelia viridis nidovirus isolate BH171/14-7, ...",VirusesRiboviriaNidoviralesunclassified Nidovi...,unk
3830,NC_040831.2,Porcine serum associated circular DNA virus 2 ...,Virusesunclassified virusesunclassified DNA vi...,unk
4333,MK795518.1,"Varroa destructor virus 3 strain USA, complete...",Virusesunclassified viruses,unk
...,...,...,...,...
57643,LN870404.1,Cotton leaf curl Multan alphasatellite complet...,VirusesSatellitesSatellite Nucleic AcidsSingle...,unk
57644,LN870405.1,Cotton leaf curl Multan alphasatellite complet...,VirusesSatellitesSatellite Nucleic AcidsSingle...,unk
57645,LN870406.1,Cotton leaf curl Multan alphasatellite complet...,VirusesSatellitesSatellite Nucleic AcidsSingle...,unk
57646,LN870407.1,Cotton leaf curl Multan alphasatellite complet...,VirusesSatellitesSatellite Nucleic AcidsSingle...,unk


In [13]:
df = df.loc[df['Gen'] != 'unk']
ai = []

for i in df['Id'].to_list():
    ai.append(eyed.index(i))

c = 0
slist = []
for i in SeqIO.parse('1.gb', 'genbank'):
    if c in ai:
        slist.append(i.seq)
    c+=1

df['Seq'] = slist

df = df.drop(columns='Tax')

df.drop(columns='Gen')

df.to_csv('1 Id Desc Tax.csv', index = False)

Repeat same for 2.gb

eyed, desc, tax = [],[],[]

for i in SeqIO.parse('2.gb', 'genbank'):
    eyed.append(i.id)
    desc.append(i.description)
    tax.append(i.annotations['taxonomy'])

df = pd.DataFrame({'Id': eyed, 'Des' : desc, 'Tax' : tax})

df = df.drop_duplicates(subset='Id')

for i in fl['Fam'].to_list():
    if 'idae' not in i:
        print(i)

for i,r in df.iterrows():
    df.loc[i,'Tax'] = ''.join(r['Tax'])

In [69]:
c = 0

for i in df['Tax'].to_list():
    if 'idae' not in i:
        print(i)
        c+=1
        
print(c)

#1104 weird entries hoping to be dropped during merging

VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirus
VirusesVirus families not assigned to an orderSinaivirusunclassified Sinaivirus
VirusesssRNA virusesssRNA positive-strand viruses, no DNA stage
V

In [70]:
df['Gen'] = 'unk'

df = df.reset_index(drop = True)

for i,r in fl.iterrows():
    for i2,r2 in df.iterrows():
        if r['Fam'].lower() in r2['Tax'].lower():
            df.loc[i2, 'Gen'] = r['Genome type'] 

In [77]:
df.loc[df['Gen'] == 'unk']

#1108 left unlabelled

,Id,Des,Tax,Gen
17,KY465697.1,"Lake Sinai virus 1 strain WA1 ORF1, RNA-depend...",VirusesVirus families not assigned to an order...,unk
18,KY465698.1,"Lake Sinai virus 1 strain WA2 ORF1, RNA-depend...",VirusesVirus families not assigned to an order...,unk
19,KY465699.1,"Lake Sinai virus 1 strain VN1 ORF1, RNA-depend...",VirusesVirus families not assigned to an order...,unk
20,KY465700.1,"Lake Sinai virus 1 strain VN2 ORF1, RNA-depend...",VirusesVirus families not assigned to an order...,unk
21,KY465701.1,"Lake Sinai virus 1 strain VN3 ORF1, RNA-depend...",VirusesVirus families not assigned to an order...,unk
...,...,...,...,...
41258,HG417072.1,Guar leaf curl alphasatellite complete sequenc...,VirusesSatellitesSatellite Nucleic AcidsSingle...,unk
41259,HG417073.1,Guar leaf curl alphasatellite complete sequenc...,VirusesSatellitesSatellite Nucleic AcidsSingle...,unk
41260,HG417074.1,Guar leaf curl alphasatellite complete sequenc...,VirusesSatellitesSatellite Nucleic AcidsSingle...,unk
41261,HG417075.1,Guar leaf curl alphasatellite complete sequenc...,VirusesSatellitesSatellite Nucleic AcidsSingle...,unk


In [78]:
df = df.loc[df['Gen'] != 'unk']

ai = []

for i in df['Id'].to_list():
    ai.append(eyed.index(i))

c = 0
slist = []

for i in SeqIO.parse('2.gb', 'genbank'):
    if c in ai:
        slist.append(i.seq)
    c+=1

df['Seq'] = slist

In [82]:
df = df.drop(columns='Tax')

In [6]:
df.to_csv('2 Id Desc Tax.csv', index = False)

Repeat for 3.gb

In [33]:
eyed, desc, tax = [],[],[]

for i in SeqIO.parse('3.gb', 'genbank'):
    eyed.append(i.id)
    desc.append(i.description)
    tax.append(i.annotations['taxonomy'])

df = pd.DataFrame({'Id': eyed, 'Des' : desc, 'Tax' : tax})

df = df.drop_duplicates(subset='Id')

for i in fl['Fam'].to_list():
    if 'idae' not in i:
        print(i)

for i,r in df.iterrows():
    df.loc[i,'Tax'] = ''.join(r['Tax'])

In [90]:
c = 0

for i in df['Tax'].to_list():
    if 'idae' not in i:
        print(i)
        c+=1
        
print(c)

#965 weird entries hoping to be dropped during merging

VirusesSatellitesSatellite Nucleic AcidsSingle stranded DNA satellitesAlphasatellitesBegomovirus-associated alphasatellitesunclassified Begomovirus-associated alphasatellites
VirusesSatellitesSatellite Nucleic AcidsSingle stranded DNA satellitesAlphasatellitesBegomovirus-associated alphasatellites
VirusesSatellitesSatellite Nucleic AcidsSingle stranded DNA satellitesAlphasatellitesBegomovirus-associated alphasatellites
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
VirusesDeltavirus
V

In [91]:
df['Gen'] = 'unk'

df = df.reset_index(drop = True)

for i,r in fl.iterrows():
    for i2,r2 in df.iterrows():
        if r['Fam'].lower() in r2['Tax'].lower():
            df.loc[i2, 'Gen'] = r['Genome type'] 

In [94]:
df.loc[df['Gen'] == 'unk']

#969 left unlabelled

,Id,Des,Tax,Gen
480,LC066301.1,"Blueberry mosaic associated virus viral cRNA, ...",VirusesssRNA virusesssRNA negative-strand viru...,unk
824,KR816012.1,Cotton leaf curl Burewala alphasatellite isola...,VirusesSatellitesSatellite Nucleic AcidsSingle...,unk
825,KR816016.1,Cotton leaf curl Multan alphasatellite isolate...,VirusesSatellitesSatellite Nucleic AcidsSingle...,unk
826,KR816017.1,Cotton leaf curl Multan alphasatellite isolate...,VirusesSatellitesSatellite Nucleic AcidsSingle...,unk
1694,KF786305.1,Hepatitis delta virus isolate HDV3_Brazil_15 H...,VirusesDeltavirus,unk
...,...,...,...,...
101460,M55042.1,"Hepatitis D virus delta antigen (HDAg) mRNA, c...",VirusesDeltavirus,unk
101463,M84917.1,Hepatitis D virus RNA sequence,VirusesRiboviriaDeltavirus,unk
101488,M64284.1,Satellite Cucumber mosaic virus (strain Ix) RNA,VirusesSatellitesRNA satellitesSingle stranded...,unk
101508,M58629.1,Hepatitis delta virus large and small antigens...,VirusesRiboviriaDeltavirus,unk


In [95]:
df = df.loc[df['Gen'] != 'unk']

ai = []

for i in df['Id'].to_list():
    ai.append(eyed.index(i))

c = 0
slist = []

for i in SeqIO.parse('3.gb', 'genbank'):
    if c in ai:
        slist.append(i.seq)
    c+=1

df['Seq'] = slist

In [12]:
df = df.drop(columns='Tax')

df.to_csv('3 Id Desc Tax.csv', index = False)

saving moltype topology and organism

In [1]:
#interesting that you can load a dataframe this way without memory error
#as opposed to directly loading it

import pandas as pd
from Bio import SeqIO
mylist = []

for chunk in  pd.read_csv('1 Id Desc Gen Seq.csv', chunksize=20000):
    mylist.append(chunk)

df = pd.concat(mylist, axis= 0)
del mylist

In [2]:
org = []
moltype = []
topol = []
eyed = []

for i in SeqIO.parse('1.gb', 'genbank'):
    org.append(i.annotations['organism'])
    moltype.append(i.annotations['molecule_type'])
    topol.append(i.annotations['topology'])
    eyed.append(i.id)

df2 = pd.DataFrame({'Id' : eyed, 'Org' : org, 'MolTyp' : moltype, 'Topol' : topol})
df2 = df2.drop_duplicates(subset = 'Id')

In [4]:
df['Complete Genome'] = 'No'
for i,r in df.iterrows():
    if 'complete genome' in r['Des'].lower():
        df.loc[i,'Complete Genome'] = 'Yes'

In [5]:
df = df.loc[df['Complete Genome'] == 'Yes']

df = pd.merge(df2, df, on = 'Id', how = 'inner')

df = df.drop_duplicates('Org')

df = df.drop(columns = 'Complete Genome')

df.to_csv('1 moltyp topol des gen seq complete genomes.csv', index = False)

Repeat for 2

In [13]:
import pandas as pd
from Bio import SeqIO
mylist = []

for chunk in  pd.read_csv('2 Id Desc Gen Seq.csv', chunksize=20000):
    mylist.append(chunk)

df = pd.concat(mylist, axis= 0)
del mylist

In [14]:
org = []
moltype = []
topol = []
eyed = []

for i in SeqIO.parse('2.gb', 'genbank'):
    org.append(i.annotations['organism'])
    moltype.append(i.annotations['molecule_type'])
    topol.append(i.annotations['topology'])
    eyed.append(i.id)

In [15]:
df2 = pd.DataFrame({'Id' : eyed, 'Org' : org, 'MolTyp' : moltype, 'Topol' : topol})
df2 = df2.drop_duplicates(subset = 'Id')

In [16]:
df['Complete Genome'] = 'No'
for i,r in df.iterrows():
    if 'complete genome' in r['Des'].lower():
        df.loc[i,'Complete Genome'] = 'Yes'

In [17]:
df = df.loc[df['Complete Genome'] == 'Yes']

df = pd.merge(df2, df, on = 'Id', how = 'inner')

df = df.drop_duplicates('Org')

df = df.drop(columns = 'Complete Genome')

df.to_csv('2 moltyp topol des gen seq complete genomes.csv', index = False)

Repeat for 3

In [1]:
import pandas as pd
from Bio import SeqIO
mylist = []

for chunk in  pd.read_csv('3 Id Desc Gen Seq.csv', chunksize=20000):
    mylist.append(chunk)

df = pd.concat(mylist, axis= 0)
del mylist

org = []
moltype = []
topol = []
eyed = []

for i in SeqIO.parse('3.gb', 'genbank'):
    org.append(i.annotations['organism'])
    moltype.append(i.annotations['molecule_type'])
    topol.append(i.annotations['topology'])
    eyed.append(i.id)

df2 = pd.DataFrame({'Id' : eyed, 'Org' : org, 'MolTyp' : moltype, 'Topol' : topol})
df2 = df2.drop_duplicates(subset = 'Id')

df['Complete Genome'] = 'No'
for i,r in df.iterrows():
    if 'complete genome' in r['Des'].lower():
        df.loc[i,'Complete Genome'] = 'Yes'

df = df.loc[df['Complete Genome'] == 'Yes']

df = pd.merge(df2, df, on = 'Id', how = 'inner')

df = df.drop_duplicates('Org')

df = df.drop(columns = 'Complete Genome')


df.to_csv('3 moltyp topol des gen seq complete genomes.csv', index = False)

combine 3 filtered data files

In [4]:
df1 = pd.read_csv('1 moltyp topol des gen seq complete genomes.csv')
df2 = pd.read_csv('2 moltyp topol des gen seq complete genomes.csv')
df3 = pd.read_csv('3 moltyp topol des gen seq complete genomes.csv')

df = pd.concat([df1,df2,df3], axis = 0)

assert len(df1) + len(df2) + len(df3) == len(df)

df = df.drop_duplicates(subset = 'Org')

df = df.reset_index(drop= True)

In [25]:
#This makes no sense to me

df.loc[df.duplicated(subset = 'Seq', keep = False)].sort_values(by = 'Seq')

,Id,Org,MolTyp,Topol,Des,Gen,Seq
220,KR492908.1,Fenneropenaeus chinensis hepandensovirus,DNA,linear,Fenneropenaeus chinensis hepandensovirus isola...,ssDNA(+/-),AAACCAGGGGGGTGTGGTAAGAGTCCTCTCCGAGTGGTCTAGCAGT...
222,KT316240.1,Decapod hepandensovirus 1,DNA,linear,"Decapod hepandensovirus 1 isolate DIBAH1, comp...",ssDNA(+/-),AAACCAGGGGGGTGTGGTAAGAGTCCTCTCCGAGTGGTCTAGCAGT...
7397,KF429789.1,Norovirus Hu/GI.1/8MC/1978/USA,ss-RNA,linear,"Norovirus Hu/GI.1/8MC/1978/USA, complete genome",ssRNA(+),AATGATGATGGCGTCAAAAGACGTCGTTCCTACTGCTGCTAGCAGT...
7395,KF429774.1,Norovirus Hu/GI.1/8UIIIf/1973/USA,ss-RNA,linear,"Norovirus Hu/GI.1/8UIIIf/1973/USA, complete ge...",ssRNA(+),AATGATGATGGCGTCAAAAGACGTCGTTCCTACTGCTGCTAGCAGT...
7392,KF429761.1,Norovirus Hu/GI.1/8MoIIIL/1972/USA,ss-RNA,linear,"Norovirus Hu/GI.1/8MoIIIL/1972/USA, complete g...",ssRNA(+),AATGATGATGGCGTCAAAAGACGTCGTTCCTACTGCTGCTAGCAGT...
...,...,...,...,...,...,...,...
7044,KJ133696.1,Brucella phage 177_19,DNA,linear,"Brucella phage 177_19, complete genome",dsDNA,TTAGCTATCTTGCTACATCTAGTTGTAGCAATTATATGACTTATAC...
7045,KJ133697.1,Brucella phage 177_141,DNA,linear,"Brucella phage 177_141, complete genome",dsDNA,TTAGCTATCTTGCTACATCTAGTTGTAGCAATTATATGACTTATAC...
7037,KJ133689.1,Brucella phage 02_141,DNA,linear,"Brucella phage 02_141, complete genome",dsDNA,TTAGCTATCTTGCTACATCTAGTTGTAGCAATTATATGACTTATAC...
7038,KJ133690.1,Brucella phage 11sa_19,DNA,linear,"Brucella phage 11sa_19, complete genome",dsDNA,TTAGCTATCTTGCTACATCTAGTTGTAGCAATTATATGACTTATAC...


In [ ]:
checking this out from actual website entries

In [30]:
a = '''        1 aaaccagggg ggtgtggtaa gagtcctctc cgagtggtct agcagtgcag tactcagcaa
       61 gatggcgagc aaaggtgaat attattattt tactcctaga gatctatgga attttgtgaa
      121 taccttgctc agtcataaga taagtactat agctatgaaa atgctgacca tgaaggagaa
      181 ggaattcttg agagaagttt ggataccgta tatacaaggt ttccagacag ctcatgactt
      241 gggtgtagat aacgataagt gcggtgcggc gggaagtgaa tacaagaaag tttttctcga
      301 tgaggagaag tttatggact ttgtgacatc tttccgccgc agcgtattta taagttatgg
      361 agatagatgg agacaggatg gatcatattt tatggcagat gtattttgga gtcatcagtt
      421 aagacagttt tggatgatga tatttggaga atgtttcaga gtaaacatgt caccatgtaa
      481 gagattgtat atagacatgt taccttatta ttatttagca aagatgacaa cagagaatag
      541 acatctagta gagtgggagt atatgaatcc atgtcctaac acacaagtaa ggaggaataa
      601 gatgactggt atgaattttt gtaagcaagg agttgtgata gataatgagt atccagacaa
      661 tcagatgggt tgttacaata tagatgaaca tccattacca ggaggtatta gatggagtgg
      721 taataacgag tatcgtacag gttatgtaca cgttaataag gtgaaatggt tgggagttgc
      781 agataaagtc agtgacatgg aggagacatc aagtgatgag gaggtgccca gcagccagga
      841 gaaatacgtg aagagcaagg agaagaagga acaacagaag acagagaaga aggaggatga
      901 gccggcgaat aagaagagga agttctcgct tactaacaca gtgttggaga agcagaagat
      961 ggagttgggg aagtttttcc gaatggaaga ggaaccaatc aacatcaagc tatacgaatt
     1021 ggaggaagga aaggaacacc atgtacatga ggctatacgt atcgatggta gtaactctaa
     1081 gttcgctaag aagaaagacg agcagggtaa tgttatggat gatttcaagg taattgtttg
     1141 tgatggagag aataacttgt atggtttttt tgcaaacaca caattgaata agttatttaa
     1201 caaaagacat ggaacaaaga aatatagtat gaaaccagaa cacaatataa gtttaaaggt
     1261 ttcacagatt tcagaggtta ggaatggaaa gatgtgtatt gtaaagatgg ctattaatga
     1321 tgatgtaaaa tgtttcgcca ggtaatgaag ttgaagagag aaggtatggt ggatcataac
     1381 cctcttgtta cattttactc tggtataatt gtaaagtttg agcattggaa taataatgtc
     1441 agtaaagtaa gaaagttttt gtataagttt gcacagtggt tgtataagga atgtacatac
     1501 atccacaata taagtgccgc agttcatgat agatgtaaag atgattgttg taaagactca
     1561 gccaataaag tatgtaagaa catatatggt cctcatttac acattttatt ggagagtgtg
     1621 aatgagaatt ggagtaagag tagcaagagg gttttattcc gcggctacga gaagatccta
     1681 cagcatgaaa acaaacagct atgggaggac ctaggactac agaagacatc acctccatca
     1741 atgagtctat gggatggtga gatgtttaag tggtacatgt tcagagatag gaaatatgca
     1801 aaggtgcatg gaacatacta ctatagtagt aatgaggaat tgttgagtaa gttagtaaag
     1861 atgaaggata cacaggagag agatgatttg tatgagaagg catgtcaatt caagaaagat
     1921 aagaatgcag caagaaagat tgagaacagt actgctaaga cattagatgg tgctgatagt
     1981 gacaacataa ggttaagcag ttctagagct atatacctag agaatttaca ggtactggag
     2041 aagtatctag taaagcataa gtgttacaca attcaagatt ttaagatgat gcaaaggaat
     2101 gaagatgaaa tttgggtgaa ttacatgtat gatatacaga atttagagaa ggtgatagag
     2161 aagttgaaca ttatggagta tagtttacaa caagcagatt acattgaggg aaatacatgg
     2221 ataggagagg atttatggaa cactaacagt gcatacatga ggactgttaa gagaggtact
     2281 gatagatact attggtatat tcagagacac gtttcaaata gagccagttt gattggacag
     2341 agcagacaga tttgtattga tggagcatat atgatgttta atatcattga caacatgaga
     2401 gtagagtcaa gaccaaagac aataccaata ataagcaaga ataagacagt acagtggata
     2461 caggatttta tggatatcat acatggaaat ttacccaaga ttaactgtat gatgttatac
     2521 gggaatagta acagtggaaa gacacaattg attgaagctt tgacaggatt ggtaaataca
     2581 gccataatga caaatgttgg tgatggtgga acattccatt tcagtaacat aacagagatg
     2641 agtacaattg tagtagggaa tgagaccaag attaggactc agacaattga gcagtggaag
     2701 ggattatgtg gaggagagaa tgtaacaatg cctatgaagt ataaggaaca caagactcat
     2761 atgttcagga agcctgtatt tttgactaat cagcatcatc cactggtaga catatcacac
     2821 tatgatgaca gaagagctat agagaataga agtttcatgt ataaggtaga gttaggaagt
     2881 gagccagtaa atgcacatat aaagtttcct aataggatga ttccaatcaa gaagaatcca
     2941 gaactaacac agtttgtatt ggccagcatg cagtatgttc atacaaatta tatgaacaga
     3001 ccagacaaga agtttaaaat tggatttttc aacaagcttt atgacgcgct gtttgagaat
     3061 agctaaatct atgtacgcag ggttcaagtt ttcccggcat aaataaagtg ataagataag
     3121 attgagagtt tgaatatcca cgtcacacac aaagatagta taccatgagt ctagcatggg
     3181 agcagtcgta tctgcagttg ctgcagtaat agctgtagta gtcgaagtcg tcgagttcat
     3241 agtcgatgtc gttgaagtgg ctttcgtggt tgcagaaact gtacaggtcg tctgcagaca
     3301 cagtcgatta ctttgttggc ggcgatagta catgcgcaga acgatcagca gcaatccgaa
     3361 gacctggtgg aaggagttaa cgaaattaac gacagaacaa catcgacgga agtaggacgt
     3421 gagcacaggg ccatatgcta ggacctgatc agctacgacg aatacctgga gagagcaaca
     3481 gaaatagact acatgtcacc tacaagaaaa ggaggaaatt attttgcaag taaacatttt
     3541 caaagtaaga agaagaataa actaggaaaa gtaaaggatc tactagcaag caagaagaag
     3601 gagagaaagt ttaaaggaaa aggaaatact ttaagtgagg aaccaagtac atcggggtgg
     3661 aaagatccag taagacagag atttccagca ttagaacaag aagagagaaa tacatttgca
     3721 ggattattgg caatagaagc agcaccggat cagagacaac taggacgtga tagtaacaac
     3781 cagttagcat tggtacagag agatacaaga gtagcagtaa gacaaagtac aaacagaaga
     3841 gaagcattag aagtagtaag agcagcaact caagcaataa gatcgggtgg agatagacta
     3901 gcagaattag tacagacata cgcatcagga ttttcggaca gtacagaaat agtagagcta
     3961 agacaagaag atagaataca aagagatata ttccaagaag aaggacaaaa cttattagct
     4021 atcgagatcg cattagaagg accaagcagt gtaacacagc aatttgatca agagaaaact
     4081 ccagcagtca agagagcttt agaactaact caagaagaag aacagttaga acggatagaa
     4141 aacgctaaga agtatattga ggaggttata gaggagacca atagagaatt tgaagatcaa
     4201 gtacgacaag agacaagtgc gtcggcggaa gatacgatgg ccagtgccgc acctactccg
     4261 atggaaacct ctgagccagg ggttaccgcc gcaccgcacc agaaatctgc agcgggcggt
     4321 ggtggcggtg gtggcagtgg aggtgagaca gcagggtatg gtaaaaatac caacgatgcc
     4381 ttccagcgcc accgcaatag acccattgat ctcaaacata ttggagacaa tgtgtatgta
     4441 gctcagaggg tttataaagt ggaggctgaa tgcaagaata tagatgagaa gttgacatgg
     4501 tcaaacacag caaacaacga atatacacga aggctgatgg gaacgaatgg aaacagcgac
     4561 aaaggtaatg ttaagttcaa ctttacagca ccgatcaatg gaagtattgg aatgggaaat
     4621 cttgctttgg gtaactatat aaatgcatgg ggtatagaca acattgcaaa gagtgaagac
     4681 agctgggcta ttatagctac aagaggcaag atgaaccatt tacaggcatt tgagatgatt
     4741 ccaccaaatg caaggagaaa ctgtagtagg gtatacaagt gctccagtac agtttggtaa
     4801 gctattggga catatatatt atccagatcc aaaaggacaa gaaaagataa agatcgcaag
     4861 cttggagaat ggcagagaat ataaggtatt caacaacgca ttgagaggat accaaatgga
     4921 cgacgacatg aatcagacaa aagtcacagc agctgacaat aaggtacatg catttacaga
     4981 cttacgagat gcaccaatga taagtcatgt agcagcgtac ctgaacaaag aaaacccacc
     5041 agcaatcaat ggtataggaa tagaatacca gggatacgac atggccaatg agggtatgtc
     5101 atctctcatt ggagtaatac caagtaactg tataaataaa aggaaggaaa tacagtcagg
     5161 tatggataat gtagtactct ggtctatgca aagtaacagg ctcatagaca aaagattctg
     5221 gacaccagaa ggatggagtg agacaagtgc aaacgatatg gctaacaaca gagtaaacgt
     5281 tcctgcagag aatggtgtaa tatatgacga agctcatgta acaagaacat cagactatgc
     5341 agaatgggcc aggaatgaga tatactacaa tgcaggtaat gtatacagtg agtttgcacc
     5401 aggtaacagt ggagcattca caagcaagta taatataagt aaccagtatg ctacacagat
     5461 tttcttcatg ccatatgctc atacacagcg aggagcgata caagatatcg tcataaactt
     5521 cgacctaaca ctacaaatta tggtcaagag gattccacgt catgtgtata atgatttcta
     5581 tcacataaac actagagcag tgggtccaat agtgtatgaa gaatacaagg aaagagcatt
     5641 tggagcagct gatgtaacta caagagctaa tgagatacat gtgaacataa gtggaactca
     5701 tggaagtaag tatgcagaca gaggacaagt atcatggaag cagtcagtaa gaaaggagtt
     5761 cgccaagaga cagtatggac aaaaaagaat aattctgaat caaggaataa acacaagaag
     5821 agtaacaaga agcagcagtg cggcggagga agatgatata ccagaagatt gtgacgactt
     5881 cctagaaact tcagaaatag agtcgccgcc gcagccgcag tcgcagaaga agaagaagaa
     5941 gtataagact attgtataat gttattgtaa tacaaatgta cataatgggt tataataaag
     6001 gttctataaa atcacgcatg cgtggtttaa atctctagag gagcaaccct cagggggtag
     6061 gtagggggat ctttgcgatg acact'''

In [32]:
b = '''        1 aaaccagggg ggtgtggtaa gagtcctctc cgagtggtct agcagtgcag tactcagcaa
       61 gatggcgagc aaaggtgaat attattattt tactcctaga gatctatgga attttgtgaa
      121 taccttgctc agtcataaga taagtactat agctatgaaa atgctgacca tgaaggagaa
      181 ggaattcttg agagaagttt ggataccgta tatacaaggt ttccagacag ctcatgactt
      241 gggtgtagat aacgataagt gcggtgcggc gggaagtgaa tacaagaaag tttttctcga
      301 tgaggagaag tttatggact ttgtgacatc tttccgccgc agcgtattta taagttatgg
      361 agatagatgg agacaggatg gatcatattt tatggcagat gtattttgga gtcatcagtt
      421 aagacagttt tggatgatga tatttggaga atgtttcaga gtaaacatgt caccatgtaa
      481 gagattgtat atagacatgt taccttatta ttatttagca aagatgacaa cagagaatag
      541 acatctagta gagtgggagt atatgaatcc atgtcctaac acacaagtaa ggaggaataa
      601 gatgactggt atgaattttt gtaagcaagg agttgtgata gataatgagt atccagacaa
      661 tcagatgggt tgttacaata tagatgaaca tccattacca ggaggtatta gatggagtgg
      721 taataacgag tatcgtacag gttatgtaca cgttaataag gtgaaatggt tgggagttgc
      781 agataaagtc agtgacatgg aggagacatc aagtgatgag gaggtgccca gcagccagga
      841 gaaatacgtg aagagcaagg agaagaagga acaacagaag acagagaaga aggaggatga
      901 gccggcgaat aagaagagga agttctcgct tactaacaca gtgttggaga agcagaagat
      961 ggagttgggg aagtttttcc gaatggaaga ggaaccaatc aacatcaagc tatacgaatt
     1021 ggaggaagga aaggaacacc atgtacatga ggctatacgt atcgatggta gtaactctaa
     1081 gttcgctaag aagaaagacg agcagggtaa tgttatggat gatttcaagg taattgtttg
     1141 tgatggagag aataacttgt atggtttttt tgcaaacaca caattgaata agttatttaa
     1201 caaaagacat ggaacaaaga aatatagtat gaaaccagaa cacaatataa gtttaaaggt
     1261 ttcacagatt tcagaggtta ggaatggaaa gatgtgtatt gtaaagatgg ctattaatga
     1321 tgatgtaaaa tgtttcgcca ggtaatgaag ttgaagagag aaggtatggt ggatcataac
     1381 cctcttgtta cattttactc tggtataatt gtaaagtttg agcattggaa taataatgtc
     1441 agtaaagtaa gaaagttttt gtataagttt gcacagtggt tgtataagga atgtacatac
     1501 atccacaata taagtgccgc agttcatgat agatgtaaag atgattgttg taaagactca
     1561 gccaataaag tatgtaagaa catatatggt cctcatttac acattttatt ggagagtgtg
     1621 aatgagaatt ggagtaagag tagcaagagg gttttattcc gcggctacga gaagatccta
     1681 cagcatgaaa acaaacagct atgggaggac ctaggactac agaagacatc acctccatca
     1741 atgagtctat gggatggtga gatgtttaag tggtacatgt tcagagatag gaaatatgca
     1801 aaggtgcatg gaacatacta ctatagtagt aatgaggaat tgttgagtaa gttagtaaag
     1861 atgaaggata cacaggagag agatgatttg tatgagaagg catgtcaatt caagaaagat
     1921 aagaatgcag caagaaagat tgagaacagt actgctaaga cattagatgg tgctgatagt
     1981 gacaacataa ggttaagcag ttctagagct atatacctag agaatttaca ggtactggag
     2041 aagtatctag taaagcataa gtgttacaca attcaagatt ttaagatgat gcaaaggaat
     2101 gaagatgaaa tttgggtgaa ttacatgtat gatatacaga atttagagaa ggtgatagag
     2161 aagttgaaca ttatggagta tagtttacaa caagcagatt acattgaggg aaatacatgg
     2221 ataggagagg atttatggaa cactaacagt gcatacatga ggactgttaa gagaggtact
     2281 gatagatact attggtatat tcagagacac gtttcaaata gagccagttt gattggacag
     2341 agcagacaga tttgtattga tggagcatat atgatgttta atatcattga caacatgaga
     2401 gtagagtcaa gaccaaagac aataccaata ataagcaaga ataagacagt acagtggata
     2461 caggatttta tggatatcat acatggaaat ttacccaaga ttaactgtat gatgttatac
     2521 gggaatagta acagtggaaa gacacaattg attgaagctt tgacaggatt ggtaaataca
     2581 gccataatga caaatgttgg tgatggtgga acattccatt tcagtaacat aacagagatg
     2641 agtacaattg tagtagggaa tgagaccaag attaggactc agacaattga gcagtggaag
     2701 ggattatgtg gaggagagaa tgtaacaatg cctatgaagt ataaggaaca caagactcat
     2761 atgttcagga agcctgtatt tttgactaat cagcatcatc cactggtaga catatcacac
     2821 tatgatgaca gaagagctat agagaataga agtttcatgt ataaggtaga gttaggaagt
     2881 gagccagtaa atgcacatat aaagtttcct aataggatga ttccaatcaa gaagaatcca
     2941 gaactaacac agtttgtatt ggccagcatg cagtatgttc atacaaatta tatgaacaga
     3001 ccagacaaga agtttaaaat tggatttttc aacaagcttt atgacgcgct gtttgagaat
     3061 agctaaatct atgtacgcag ggttcaagtt ttcccggcat aaataaagtg ataagataag
     3121 attgagagtt tgaatatcca cgtcacacac aaagatagta taccatgagt ctagcatggg
     3181 agcagtcgta tctgcagttg ctgcagtaat agctgtagta gtcgaagtcg tcgagttcat
     3241 agtcgatgtc gttgaagtgg ctttcgtggt tgcagaaact gtacaggtcg tctgcagaca
     3301 cagtcgatta ctttgttggc ggcgatagta catgcgcaga acgatcagca gcaatccgaa
     3361 gacctggtgg aaggagttaa cgaaattaac gacagaacaa catcgacgga agtaggacgt
     3421 gagcacaggg ccatatgcta ggacctgatc agctacgacg aatacctgga gagagcaaca
     3481 gaaatagact acatgtcacc tacaagaaaa ggaggaaatt attttgcaag taaacatttt
     3541 caaagtaaga agaagaataa actaggaaaa gtaaaggatc tactagcaag caagaagaag
     3601 gagagaaagt ttaaaggaaa aggaaatact ttaagtgagg aaccaagtac atcggggtgg
     3661 aaagatccag taagacagag atttccagca ttagaacaag aagagagaaa tacatttgca
     3721 ggattattgg caatagaagc agcaccggat cagagacaac taggacgtga tagtaacaac
     3781 cagttagcat tggtacagag agatacaaga gtagcagtaa gacaaagtac aaacagaaga
     3841 gaagcattag aagtagtaag agcagcaact caagcaataa gatcgggtgg agatagacta
     3901 gcagaattag tacagacata cgcatcagga ttttcggaca gtacagaaat agtagagcta
     3961 agacaagaag atagaataca aagagatata ttccaagaag aaggacaaaa cttattagct
     4021 atcgagatcg cattagaagg accaagcagt gtaacacagc aatttgatca agagaaaact
     4081 ccagcagtca agagagcttt agaactaact caagaagaag aacagttaga acggatagaa
     4141 aacgctaaga agtatattga ggaggttata gaggagacca atagagaatt tgaagatcaa
     4201 gtacgacaag agacaagtgc gtcggcggaa gatacgatgg ccagtgccgc acctactccg
     4261 atggaaacct ctgagccagg ggttaccgcc gcaccgcacc agaaatctgc agcgggcggt
     4321 ggtggcggtg gtggcagtgg aggtgagaca gcagggtatg gtaaaaatac caacgatgcc
     4381 ttccagcgcc accgcaatag acccattgat ctcaaacata ttggagacaa tgtgtatgta
     4441 gctcagaggg tttataaagt ggaggctgaa tgcaagaata tagatgagaa gttgacatgg
     4501 tcaaacacag caaacaacga atatacacga aggctgatgg gaacgaatgg aaacagcgac
     4561 aaaggtaatg ttaagttcaa ctttacagca ccgatcaatg gaagtattgg aatgggaaat
     4621 cttgctttgg gtaactatat aaatgcatgg ggtatagaca acattgcaaa gagtgaagac
     4681 agctgggcta ttatagctac aagaggcaag atgaaccatt tacaggcatt tgagatgatt
     4741 ccaccaaatg caaggagaaa ctgtagtagg gtatacaagt gctccagtac agtttggtaa
     4801 gctattggga catatatatt atccagatcc aaaaggacaa gaaaagataa agatcgcaag
     4861 cttggagaat ggcagagaat ataaggtatt caacaacgca ttgagaggat accaaatgga
     4921 cgacgacatg aatcagacaa aagtcacagc agctgacaat aaggtacatg catttacaga
     4981 cttacgagat gcaccaatga taagtcatgt agcagcgtac ctgaacaaag aaaacccacc
     5041 agcaatcaat ggtataggaa tagaatacca gggatacgac atggccaatg agggtatgtc
     5101 atctctcatt ggagtaatac caagtaactg tataaataaa aggaaggaaa tacagtcagg
     5161 tatggataat gtagtactct ggtctatgca aagtaacagg ctcatagaca aaagattctg
     5221 gacaccagaa ggatggagtg agacaagtgc aaacgatatg gctaacaaca gagtaaacgt
     5281 tcctgcagag aatggtgtaa tatatgacga agctcatgta acaagaacat cagactatgc
     5341 agaatgggcc aggaatgaga tatactacaa tgcaggtaat gtatacagtg agtttgcacc
     5401 aggtaacagt ggagcattca caagcaagta taatataagt aaccagtatg ctacacagat
     5461 tttcttcatg ccatatgctc atacacagcg aggagcgata caagatatcg tcataaactt
     5521 cgacctaaca ctacaaatta tggtcaagag gattccacgt catgtgtata atgatttcta
     5581 tcacataaac actagagcag tgggtccaat agtgtatgaa gaatacaagg aaagagcatt
     5641 tggagcagct gatgtaacta caagagctaa tgagatacat gtgaacataa gtggaactca
     5701 tggaagtaag tatgcagaca gaggacaagt atcatggaag cagtcagtaa gaaaggagtt
     5761 cgccaagaga cagtatggac aaaaaagaat aattctgaat caaggaataa acacaagaag
     5821 agtaacaaga agcagcagtg cggcggagga agatgatata ccagaagatt gtgacgactt
     5881 cctagaaact tcagaaatag agtcgccgcc gcagccgcag tcgcagaaga agaagaagaa
     5941 gtataagact attgtataat gttattgtaa tacaaatgta cataatgggt tataataaag
     6001 gttctataaa atcacgcatg cgtggtttaa atctctagag gagcaaccct cagggggtag
     6061 gtagggggat ctttgcgatg acact'''

In [33]:
a == b

True

In [ ]:
They ARE the same. wtf xD

In [36]:
df = df.drop_duplicates(subset = 'Seq')

df.to_csv('final file hopefully no errors.csv', index = False)

Some stats

In [43]:
set(df['Gen'].to_list())

{'dsDNA',
 'dsDNA-RT',
 'dsDNA; ssDNA',
 'dsRNA',
 'ssDNA',
 'ssDNA(+)',
 'ssDNA(+/-)',
 'ssDNA(-)',
 'ssRNA',
 'ssRNA(+)',
 'ssRNA(+/-)',
 'ssRNA(-)',
 'ssRNA-RT'}

In [45]:
df.groupby('Gen')['Id'].nunique()

Gen
dsDNA           3781
dsDNA-RT         138
dsDNA; ssDNA       8
dsRNA            148
ssDNA            276
ssDNA(+)         130
ssDNA(+/-)       941
ssDNA(-)          99
ssRNA             54
ssRNA(+)        2862
ssRNA(+/-)        24
ssRNA(-)         283
ssRNA-RT         128
Name: Id, dtype: int64

In [46]:
df.groupby('Topol')['Id'].nunique()

Topol
circular    2327
linear      6545
Name: Id, dtype: int64

Preparing files for MSA and phylogenetics

In [3]:
df = pd.read_csv('final file hopefully no errors.csv')

In [26]:
dfp = df.loc[df['Gen'] == 'ssRNA(+)']

dfn = df.loc[df['Gen'] == 'ssRNA(-)']

dfpn = df.loc[df['Gen'] == 'ssRNA(+/-)']

dfd = df.loc[df['Gen'] == 'dsRNA']

dfrt = df.loc[df['Gen'] == 'ssRNA-RT']

dfssr = df.loc[df['Gen'] == 'ssRNA']

In [28]:
for i,r in dfp.iterrows():
    with open('p, n and pn.fasta', 'a') as file:
        file.write('> %s | %s | %s | %s' % (r['Id'], r['Org'], r['Des'], r['Gen']))
        file.write('\n')
        file.write(r['Seq'])
        file.write('\n')
        
for i,r in dfn.iterrows():
    with open('p, n and pn.fasta', 'a') as file:
        file.write('> %s | %s | %s | %s' % (r['Id'], r['Org'], r['Des'], r['Gen']))
        file.write('\n')
        file.write(r['Seq'])
        file.write('\n')
    
for i,r in dfpn.iterrows():
    with open('p, n and pn.fasta', 'a') as file:
        file.write('> %s | %s | %s | %s' % (r['Id'], r['Org'], r['Des'], r['Gen']))
        file.write('\n')
        file.write(r['Seq'])
        file.write('\n')

        
for i,r in dfd.iterrows():
    with open('p, n and pn.fasta', 'a') as file:
        file.write('> %s | %s | %s | %s' % (r['Id'], r['Org'], r['Des'], r['Gen']))
        file.write('\n')
        file.write(r['Seq'])
        file.write('\n')
        
for i,r in dfr.iterrows():
    with open('p, n and pn.fasta', 'a') as file:
        file.write('> %s | %s | %s | %s' % (r['Id'], r['Org'], r['Des'], r['Gen']))
        file.write('\n')
        file.write(r['Seq'])
        file.write('\n')
        
for i,r in dfssr.iterrows():
    with open('p, n and pn.fasta', 'a') as file:
        file.write('> %s | %s | %s | %s' % (r['Id'], r['Org'], r['Des'], r['Gen']))
        file.write('\n')
        file.write(r['Seq'])
        file.write('\n')

In [33]:
c = 0

i = []
for sr in SeqIO.parse('p, n and pn.fasta', 'fasta'):
    i.append(sr.id)    
    c+=1
    
assert c == len(dfp) + len(dfn) + len(dfpn) + len(dfd) + len(dfr) + len(dfssr)

In [38]:
#clean misclassifications

3499

In [15]:
df = pd.read_csv('../final file hopefully no errors.csv')

In [5]:
df.loc[df['Gen'] == 'ssRNA(+)'].loc[df['MolTyp'] == 'DNA']

#human enterovirus c105 is an RNA virus
#turnip yellow virus is an RNA virus. Polerovirus is a genus of +ssRNA
#soybean dwarf virus is a member of luteoviridae, a genus of +ssRNA
#rest belong to +ssRNA families.

,Id,Org,MolTyp,Topol,Des,Gen,Seq
1185,MH229997.1,Human enterovirus C105,DNA,linear,"Human enterovirus C105 isolate R2759, complete...",ssRNA(+),TTCCCACCCCTAGGATCCACTGGATGTCAGTACACTGGTATCGCGG...
1478,LR584019.1,Turnip yellows virus,DNA,linear,"Turnip yellows virus genome assembly, complete...",ssRNA(+),AAGAGCGCAACAATGAATTGTTCGATGTTAAGCAAAAGACCTGCTA...
1479,LR584028.1,Soybean dwarf virus,DNA,linear,"Soybean dwarf virus genome assembly, complete ...",ssRNA(+),CACCTTTACAGAAGCGATCTTGCAAGCAAGAGTTAATTCCTTGAGA...
8537,AY646855.2,Sapovirus SaKaeo-15/Thailand,DNA,linear,"Sapovirus SaKaeo-15/Thailand, complete genome",ssRNA(+),GTGATTGGTTAGTATGGCTTCCAAGCCATTCTACCCAATGGAGTTC...
8846,AF123432.1,Bovine enterovirus strain K2577,DNA,linear,"Bovine enterovirus isolate K2577, complete genome",ssRNA(+),TTAAAACAGCCTGGGGGTTGTACCCACCCCTGGGGCCCACGCGGCG...
8847,AF123433.1,Bovine enterovirus strain SL305,DNA,linear,"Bovine enterovirus isolate SL305, complete genome",ssRNA(+),TTAAAACAGCCTGGGGGTTGTACCCACCCCCGGGGCCCACGTGGCG...


In [6]:
df.loc[df['Gen'] == 'ssRNA(-)'].loc[df['MolTyp'] == 'DNA']

#assuming citrus psorosis virus is properly classified 

,Id,Org,MolTyp,Topol,Des,Gen,Seq
2967,NC_006314.1,Citrus psorosis virus,DNA,linear,"Citrus psorosis virus RNA1, complete genome",ssRNA(-),GGTATCCTAATTAGGATTATCGTCTTGTTAGACAATTGATAGCGCA...


In [1]:
df.loc[df['Gen'] == 'ssRNA(+/-)'].loc[df['MolTyp'] == 'DNA']

NameError: name 'df' is not defined

In [ ]:
#saving just id desc tax

In [7]:
d1 = pd.read_csv('1 Id Desc Tax.csv')
d2 = pd.read_csv('2 Id Desc Tax.csv')
d3 = pd.read_csv('3 Id Desc Tax.csv')

d = pd.concat([d1,d2,d3])

d = d.drop_duplicates(subset='Id')
d = d.drop(columns = 'Gen')

d.to_csv('id desc tax.csv')

export pndfn

In [16]:
df = pd.read_csv('../final file hopefully no errors.csv')
dfp = df.loc[df['Gen'] == 'ssRNA(+)']
dfn = df.loc[df['Gen'] == 'ssRNA(-)']

In [17]:
df = pd.concat([dfp,dfn], axis = 0).reset_index(drop = True)

In [11]:
a = Seq(df.loc[2862,'Seq'], IUPAC.ambiguous_dna)
a = str(a.complement())
a = a[::-1]
dfn2 = df[2862:].reset_index(drop = True)

for i,r in dfn2.iterrows():
    sq = Seq(r['Seq'], IUPAC.ambiguous_dna)
    dfn2.loc[i,'Seq'] = sq.reverse_complement()
    
assert str(dfn2.loc[0,'Seq']) == a

In [12]:
for i,r in dfn2.iterrows():
    dfn2.loc[i,'Seq'] = str(r['Seq'])

In [24]:
pndfn = pd.concat([df,dfn2], axis = 0).reset_index(drop = True)

In [26]:
pndfn.to_csv('pndfn.csv', index = False)